In [1]:
import numpy as np
import time
import requests
import os
import pickle
from lxml import html
import numpy as np
import urllib.request
import face_recognition
import cv2
import matplotlib.pyplot as plt
import pytesseract
from googlesearch import search
from PIL import Image, ImageStat
import time
tt = time.time
st = tt()

In [2]:
# get prof name from thumbnail
def save_as_pickle(file_path, file):
    with open(file_path, 'wb') as f:
        pickle.dump(file, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        file = pickle.load(f)
    return file

def plot_image_list(img_list, figsize=(12, 12), subplot_n_cols=2):
    plt.figure(figsize=(12, 12))
    for i,img in enumerate(img_list):
        plt.subplot(int(len(img_list)/subplot_n_cols)+1, subplot_n_cols, i+1)
        try:
            plt.imshow(img)
        except:
            pass
        
def brightness(img):
    img = Image.fromarray(img)
    img = img.convert('L')
    stat = ImageStat.Stat(img)
    return stat.rms[0]

def is_valid_img(img, lthresh = 100, hthresh = 200):
    bright = brightness(img)
    print(bright)
    if bright<lthresh or bright>hthresh:
        return False
    else:
        return True
        
def get_extension_files(path_to_json, ext='.json'):
    json_files = [os.path.join(path_to_json, pos_json) for pos_json in os.listdir(path_to_json) if pos_json.endswith(ext)]
    return json_files

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def post_processing(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm_img = cv2.normalize(img, norm_img, 0, 255,
                             cv2.NORM_MINMAX)
    return img

In [16]:
base_dir = 'C:\\Users\\YASH\\nptel_face_recognition\\data'
known_faces_dir = os.path.join(base_dir, 'known_faces')
full_vedios_dir = os.path.join(base_dir, 'full_vedio')

images_dir_list = [x[0] for x in os.walk(known_faces_dir)][1:]
known_face_names = [w.split('\\')[-1] for w in images_dir_list]
known_face_images_nested_list = []
for image_dir in images_dir_list:
    known_images_path_list = get_extension_files(image_dir, ext='.jpg') 
    known_face_images_nested_list.append([face_recognition.load_image_file(w) for w in known_images_path_list])

filter_idx_list=[]
known_face_encodings=[]
known_face_seprate_encodings=[]
for i,(n,nl) in enumerate(zip(known_face_names, known_face_images_nested_list)):
    enc_list = []
    for img in nl:
        enc = face_recognition.face_encodings(img)
        if enc!=[]:
            enc_list.append(enc[0])
    
    seprate_img_arr = np.array(enc_list)
    img_arr = np.array(enc_list).mean()
    if enc_list!=[]:
        known_face_encodings.append(img_arr)
        known_face_seprate_encodings.append(seprate_img_arr)
    else:
        print(f"encoding not found for {n}")
        filter_idx_list.append(i)
known_face_names = [known_face_names[i] for i in range(len(known_face_names)) if i not in filter_idx_list]


In [7]:

tracebacks_dir = os.path.join(base_dir, "tracebacks")
os.makedirs(tracebacks_dir, exist_ok = True)

pkl_files_dir = os.path.join(base_dir, "pkl_files")
nptel_channel_name_list = load_from_pickle(os.path.join(pkl_files_dir, "nptel_channel_name_list.pkl"))
playlist_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"))

num_channels = 2
num_playlists = 2
num_vedios = 2
    
# num_channels = 5
# num_playlists = 5
# num_vedios = 5

num_frames_per_sec = 1

# for nptel_channel_name in nptel_channel_name_list:
for nptel_channel_name in nptel_channel_name_list[:num_channels]:
    chanel_pkl_files_dir = os.path.join(pkl_files_dir, nptel_channel_name)
    facef_playlist_names_list = load_from_pickle(os.path.join(chanel_pkl_files_dir, "facef_playlist_names_list.pkl"))

    chanel_vedio_dir = os.path.join(full_vedios_dir, nptel_channel_name)
    paylist_path_list = [x[0] for x in os.walk(chanel_vedio_dir)][1:]
    paylist_name_list = [x.split('\\')[-1] for x in paylist_path_list] 
    
    vedio_ext = '.mp4'
    
    namef=0
    namenf=0
    namef_playlist_names_list=[]
    namenf_playlist_names_list=[]
    # for playlist, playlistn in zip(paylist_path_list, paylist_name_list):
    for playlist, playlistn in zip(paylist_path_list[:num_playlists], paylist_name_list[:num_playlists]):
#         if playlistn not in facef_playlist_names_list:
#             continue
        true_name = playlist_2_prof_name_dict[playlistn]
        if true_name is None:
            print(f"no name for {playlistn}")
            continue
        
        vedio_path_list = get_extension_files(playlist, ext='.mp4')
        vedio_name_list = [x.split('\\')[-1] for x in vedio_path_list] 
        
        names_list = []
        for vedio_path in vedio_path_list[:num_vedios]:
            vname = vedio_path.split('\\')[-1]
            video_capture = cv2.VideoCapture(vedio_path)

            # Initialize some variables
            face_locations = []
            face_encodings = []
            face_names = []
            process_this_frame = True

            vst = 0
            vet = 30
            fps = int(video_capture.get(cv2.CAP_PROP_FPS))
            st = tt()
            i=-1
            y_pred=[]
            while video_capture.isOpened():
                i+=1
                if int(i/fps) < vst:
                    ret, frame = video_capture.read()
                    continue

                if int(i/fps) > vet:
                    break

                 # Grab a single frame of video
                ret, frame = video_capture.read()

                # Resize frame of video to 1/4 size for faster face recognition processing
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

                # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
                rgb_small_frame = small_frame[:, :, ::-1]

                # process
                if i%int(fps/num_frames_per_sec)==0:
#                     print(f"time: {int(i/fps)}")
                    # Find all the faces and face encodings in the current frame of video
                    face_locations = face_recognition.face_locations(rgb_small_frame)
                    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

                    face_names = []
                    for face_encoding in face_encodings:
                        # See if the face is a match for the known face(s)
                        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                        name = "Unknown"

                        # use the known face with the smallest distance to the new face
                        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                        best_match_index = np.argmin(face_distances)
                        if matches[best_match_index]:
                            name = known_face_names[best_match_index]
                        
                        names_list.append(name)
                        if name==true_name:
                            y_pred.append(1)
                        else:
                            y_pred.append(0)

                # Hit 'q' on the keyboard to quit!
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            # Release handle to the webcam
            video_capture.release()
            cv2.destroyAllWindows()
            print("unique predicted value counts:")
            unique, counts = np.unique(np.array(names_list), return_counts=True)
            print(np.asarray((unique, counts)).T)
            print(f"for professor:{true_name}, channel:{nptel_channel_name}, playlist:{playlistn}, vedio: {vname}\
            \n accuracy:{np.array(y_pred).mean()}\n\n")


unique predicted value counts:
[]
for professor:prof. t.k.sengupta, channel:Aerospace Engineering, playlist:Foundation of Scientific Computing, vedio: MkiUBJcgdUY.mp4            
 accuracy:nan




C:\Users\YASH\anaconda3\lib\site-packages\ipykernel_launcher.py:121: RuntimeWarning: Mean of empty slice.


unique predicted value counts:
[]
for professor:prof. t.k.sengupta, channel:Aerospace Engineering, playlist:Foundation of Scientific Computing, vedio: vT40FWuvIB8.mp4            
 accuracy:nan


no name for Instability and Transition of Fluid Flows
unique predicted value counts:
[['prof. mainak das' '16']]
for professor:prof. mainak das, channel:Biotechnology, playlist:Bio electricity, vedio: FXPDVHl4Hl4.mp4            
 accuracy:1.0


unique predicted value counts:
[['prof. mainak das' '32']]
for professor:prof. mainak das, channel:Biotechnology, playlist:Bio electricity, vedio: QiwxdcckPGc.mp4            
 accuracy:1.0


unique predicted value counts:
[]
for professor:dr. p. gopinath , channel:Biotechnology, playlist:Biomedical nanotechnology, vedio: AEWqAcSeQm4.mp4            
 accuracy:nan


unique predicted value counts:
[]
for professor:dr. p. gopinath , channel:Biotechnology, playlist:Biomedical nanotechnology, vedio: k61wjab7iUs.mp4            
 accuracy:nan




In [ ]:
# # for 1 vedio

# vedio_path = 'C:\\Users\\YASH\\nptel_face_recognition\\full_vedio\\Introduction to Machine Learning\\dYMCwxgl3vk.mp4'
# vfname = vedio_path.split('\\')[-1]
# print(f"vedio: {vfname}")
# video_capture = cv2.VideoCapture(vedio_path)

# # Initialize some variables
# face_locations = []
# face_encodings = []
# face_names = []
# process_this_frame = True

# vst = 20
# vet = 25
# fps = int(video_capture.get(cv2.CAP_PROP_FPS))
# st = tt()
# i=-1
# img_list = []
# while video_capture.isOpened():
#     i+=1
#     if int(i/fps) < vst:
#         ret, frame = video_capture.read()
#         continue

#     if int(i/fps) > vet:
#         break

#      # Grab a single frame of video
#     ret, frame = video_capture.read()

#     # Resize frame of video to 1/4 size for faster face recognition processing
#     small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

#     # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
#     rgb_small_frame = small_frame[:, :, ::-1]

#     # process
#     NUM_FRAMES_PER_SEC = 1
#     if i%int(fps/NUM_FRAMES_PER_SEC)==0:
#         print(f"time: {int(i/fps)}")
#         # Find all the faces and face encodings in the current frame of video
#         rgb_small_frame = post_processing(rgb_small_frame)
#         img_list.append(rgb_small_frame)
#         face_locations = face_recognition.face_locations(rgb_small_frame)
#         face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
#         print(f"number of faces found: {len(face_locations)}")

#         face_names = []
#         for face_encoding in face_encodings:
#             # See if the face is a match for the known face(s)
#             matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
#             name = "Unknown"

#             # # If a match was found in known_face_encodings, just use the first one.
#             # if True in matches:
#             #     first_match_index = matches.index(True)
#             #     name = known_face_names[first_match_index]

#             # Or instead, use the known face with the smallest distance to the new face
#             face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
#             best_match_index = np.argmin(face_distances)
#             if matches[best_match_index]:
#                 name = known_face_names[best_match_index]

#             face_names.append(name)

#         # Display the results
#         for (top, right, bottom, left), name in zip(face_locations, face_names):
#             # Scale back up face locations since the frame we detected in was scaled to 1/4 size
#             top *= 4
#             right *= 4
#             bottom *= 4
#             left *= 4

#             # Draw a box around the face
#             cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

#             # Draw a label with a name below the face
#             cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
#             font = cv2.FONT_HERSHEY_DUPLEX
#             cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

#         # Display the resulting image
#         cv2.imshow('Video', frame)

#     # Hit 'q' on the keyboard to quit!
#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break

# # Release handle to the webcam
# video_capture.release()
# cv2.destroyAllWindows()